<a href="https://colab.research.google.com/github/shinnew9/Apziva_practice_code/blob/main/Project3-PotentialTalents/grokai_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Try different models (to get exposed to pprl LLMs):
- phi model from Microsoft (get exposed to pplr LLMs),
- qween model (from HuggingFace) can be done (from Transformer library).
! Stay on compute version of when using google Colab. (until next time, try on Instruct base model!)

2. Another thing to try:
- LLM framework called LangChain, I can make the model to be loaded using HuggingFace spaces, but still these are expensive (Maybe I can't use very large version; they'll make me to subscribe with high pay - check on it)

Next steps would be finetuning those LLMs and then RAG.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### Necessary Package Installation

In [ ]:
!pip install transformers torch

'import warnings' failed; traceback:
Traceback (most recent call last):
  File "/usr/lib/python3.11/warnings.py", line 570, in <module>
    _processoptions(sys.warnoptions)
  File "/usr/lib/python3.11/warnings.py", line 209, in _processoptions
    _setoption(arg)
  File "/usr/lib/python3.11/warnings.py", line 225, in _setoption
    import re
  File "/usr/lib/python3.11/re/__init__.py", line 124, in <module>
    import enum
  File "/usr/lib/python3.11/enum.py", line 5, in <module>
    from functools import reduce
  File "/usr/lib/python3.11/functools.py", line 18, in <module>
    from collections import namedtuple
  File "/usr/lib/python3.11/collections/__init__.py", line 32, in <module>
    from itertools import chain as _chain
KeyboardInterrupt
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 77.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 62.2 MB/s eta 0:00:00
   ━━━

In [ ]:
!pip install --upgrade openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 4.6 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.61.1
    Uninstalling openai-1.61.1:
      Successfully uninstalled openai-1.61.1


In [ ]:
# open data
import re
import time
import openai
import numpy as np
import pandas as pd

# text data vectorization and word embedding
from sklearn.feature_extraction.text import TfidfVectorizer
# from sentence_transformers import SentenceTransfomrer

### Open CSV

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Apziva/3rd_PotentialTalents/data.csv')
df_copy = df.copy()
df_copy

,id,job_title,location,connection,fit
0,1,2019 C.T. Bauer College of Business Graduate (...,"Houston, Texas",85,NaN
1,2,Native English Teacher at EPIK (English Progra...,Kanada,500+,NaN
2,3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN
3,4,People Development Coordinator at Ryan,"Denton, Texas",500+,NaN
4,5,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+,NaN
...,...,...,...,...,...
99,100,Aspiring Human Resources Manager | Graduating ...,"Cape Girardeau, Missouri",103,NaN
100,101,Human Resources Generalist at Loparex,"Raleigh-Durham, North Carolina Area",500+,NaN
101,102,Business Intelligence and Analytics at Travelers,Greater New York City Area,49,NaN
102,103,Always set them up for Success,Greater Los Angeles Area,500+,NaN


### GrokAi-API

In [ ]:
from google.colab import userdata
userdata.get("openai_key")

response = openai.chat.completions.create(
    model = "grok-1",
    messages=[{"role": "system", "content": prompt}]
)

print(response)

ChatCompletion(id='chatcmpl-B7bKJwIxj8EzK65LZvM3ZLy3G64Hv', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello! How can I assist you today?', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1741151479, model='gpt-4-0613', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=10, prompt_tokens=9, total_tokens=19, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))


In [ ]:
def clean_connection_value(value):
  if isinstance(value, str) and "500+" in value:
    return 500  # Convert "500+" to integer 500
  try:
    return int(value)  # Convert valid numbers
  except ValueError:
    return 0  # If it fails, default to 0


df_copy["connection"] = df_copy["connection"].apply(clean_connection_value)

In [ ]:
# OpenAI API request function
def get_fitness_scores_batch(job_titles, search_term):
  """Call Grok API to determine fitness scores for candidates."""

  api_url =




  # Construct prompt for LLM
  prompt = f"""
  Given the job titles of candidates and the search term,
    determine how fit this candidates are based on the similarity to the search term.
    Assign a fitness score between 0 to 1 based on their cosine similarity.
    Return the candidates' job titles in a descending order based on the fitness score.
    Search term = {search_term},
    job titles = {job_titles}
  """

  try:
    response = openai.ChatCompletion.create(
        model = "gpt-4",     # , # text-embedding-ada-002, # GPT model choosing
        messages = [{"role":"system", "content":prompt}],
        temperature=0.2,
        max_tokens= 1000     # Allow more tokens to get the full ranked list
    )

    # Extract the response text
    response_text = response.choices[0].message.content.strip()

    # Extract job titles and scores from the response
    ranked_candidates = []
    for line in response_text.split("\n"):
      parts = line.strip().split(":")
      if len(parts) == 2:
        try:
          title, score = parts[0].strip(), float(parts[1].strip())
          ranked_candidates.append((title, max(0, min(1, score))))  # Ensure scores
        except ValueError:
          continue

    return ranked_candidates

  except Exception as e:
    print(f"Error processing batch: {e}")
    return [(title, 0) for title in job_titles]  # Default to 0 if API call fails



# Process the entire
search_term = "Aspiring human resources"   # 변경 가능
job_titles_list = df_copy["job_title"].tolist()

# Call OpenAI API (Process all )
ranked_results = get_fitness_scores_batch(job_titles_list, search_term)

# Map API results back to the dataset
fit_scores = {title: score for title, score in ranked_results}
df_copy['fit'] = df_copy['job_title'].map(fit_scores)

# Fix: Ensure fit column has no NaN values
df_copy = df_copy.assign(fit=df_copy['fit'].fillna(0))

# Sort candidates by descending fit score
df_sorted = df_copy.sort_values(by="fit", ascending=False)
df_sorted.head(5)

Error processing batch: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742



,id,job_title,location,connection,fit
0,1,2019 C.T. Bauer College of Business Graduate (...,"Houston, Texas",85,0
1,2,Native English Teacher at EPIK (English Progra...,Kanada,500,0
76,77,Human Resources|\nConflict Management|\nPolici...,Dallas/Fort Worth Area,409,0
75,76,Aspiring Human Resources Professional | Passio...,"New York, New York",212,0
74,75,"Nortia Staffing is seeking Human Resources, Pa...","San Jose, California",500,0


Next step is to get expose to open source llms (shared on HF),
- start with sLMs and then move to LLMs
(mistral, phi, qwen, and Llama, DeepSeek)


- grok API, also try APIs
- LangChain and HF spaces, allow me to work on the spaces

In [ ]:
def get_fitness_score(job_titles, search_term):
  prompt = f"""
  Given the job titles of candidates and the search term,
    determine how fit this candidates are based on the similarity to the search term
    Assign a fitness score between 0 to 1 based on their cosine similarity.
    Return the candidates' job titles in a descending order based on the fitness score.
    Search term = {search_term},
    job titles = {job_titles}
  """

  try:
    response = openai.ChatCompletion.create(
        model = "gpt-4",  # GPT model choosing
        messages = [{"role":"user", "content":prompt}],
        temperature=0.2,   # 낮은 온도로 예측안정화
        max_tokens= 10     # 숫자 하나만 반환하도록 제한
    )
    score_text = response["choices"][0]["message"]["content"].strip()
    score = float(score_text)
    return max(0, min(1, score))  # 값이 0~1 범위 내에 있도록 보정
  except Exception as e:
    print(f"Error processing: {job_titles}, {search_term} - {e}")
    return 0      # 에러 발생 시 기본값 0 반환



# Modifying these part
# it's going to be more costly, calling API every time when it works on each row, instead call the df and make it calculate the whole at once
# 각 후보자의 'fit' 점수예측,
df_copy["fit"] = df_copy.apply(lambda row: get_fitness_score(row["job_title"], row["connection"]), axis=1)

# 내림차순 정렬
df_sorted = df_copy.sort_values(by="fit", ascending=False)
df_sorted

Error processing: 2019 C.T. Bauer College of Business Graduate (Magna Cum Laude) and aspiring Human Resources professional, 85 - 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742

Error processing: Native English Teacher at EPIK (English Program in Korea), 500 - 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old

,id,job_title,location,connection,fit
0,1,2019 C.T. Bauer College of Business Graduate (...,"Houston, Texas",85,0
1,2,Native English Teacher at EPIK (English Progra...,Kanada,500,0
76,77,Human Resources|\nConflict Management|\nPolici...,Dallas/Fort Worth Area,409,0
75,76,Aspiring Human Resources Professional | Passio...,"New York, New York",212,0
74,75,"Nortia Staffing is seeking Human Resources, Pa...","San Jose, California",500,0
...,...,...,...,...,...
31,32,Native English Teacher at EPIK (English Progra...,Kanada,500,0
30,31,2019 C.T. Bauer College of Business Graduate (...,"Houston, Texas",85,0
29,30,Seeking Human Resources Opportunities,"Chicago, Illinois",390,0
28,29,Aspiring Human Resources Management student se...,"Houston, Texas Area",500,0
